# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [1]:
%store -r numerosKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[2]
print numerosKalaba
%store numerosKalaba

[2]
Stored 'numerosKalaba' (list)


In [4]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"20-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]

In [5]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [6]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/20-K1/
VER : Temps Pers
NOM : Genre Nombre
DET : Genre Nombre
ADJ : Genre Nombre
head stems
head stems,VER
head stems,NOM


In [7]:
getExemple(stems["PREP"])

'ENTRE'

In [8]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

gros
villageoise.F
ENTRE


In [9]:
paradigmes.getSigmas(["DET"])

[u'DET, Genre=M, Nombre=Sg',
 u'DET, Genre=M, Nombre=Du',
 u'DET, Genre=M, Nombre=Pl',
 u'DET, Genre=F, Nombre=Sg',
 u'DET, Genre=F, Nombre=Du',
 u'DET, Genre=F, Nombre=Pl',
 u'DET, Genre=N, Nombre=Sg',
 u'DET, Genre=N, Nombre=Du',
 u'DET, Genre=N, Nombre=Pl']

In [10]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [11]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')
regles.getRules("VER","Pers=3Pau")

[]

In [12]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
#test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

Nombre,Du,Pl,Sg
Genre,,,
F,RAD-u-s,RAD-u-t,RAD-u
M,RAD-e-s,RAD-e-t,RAD-e
N,RAD-a-s,RAD-a-t,RAD-a


In [13]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [14]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

envahir.V1 keZuNup
transformer.V2 sevuT
acheter.V1 bagan
sortir.V1 Dadejar
souffrance.F TiledaS
donner.V2 zibudel
rouge rumiT
Nicole.F nikol
noir kizow
être.V2 toZumet
quatre sejin
coup.N fezil
colère.F Sagobul
différent Sozup
sur zeDun
vers tan
apporter.V2 NabiS
ENTRE DiNe
serpent.F pevur
de kum
cuisine.N lunegik
bas baZowaN
enfant.hyper.F sadimel
villageois.M NoTeN
lance.N DotoSut
blessé wibam
loup.M TeniS
chef.F mugat
disparaître.V1 sowuD
lune.M benov
souris.M palek
Violette.F vjolet
ombre.N bibuS
enfant.M duZen
furieux poleS
rêvasser.V1 nivap
fuir.V1 deven
profond jemibot
maigre gadin
DEM b
cri.N mubeD
chatte.F piZov
chasseur.M soNom
DEF l
boire.V1 Tizok
thé.M judog
arriver.V1 poSob
nuit.M vidon
courageux joZogiN
chercher.V2 NunaDas
poisson.M wunozol
coussin.F gaZur
infirmière.F dozut
jeter.V2 gumaZ
dévorer.V1 mawes
parler.V2 Zebuf
fruit.N DomeZ
sauter.V1 Neranuf
village.N tuvej
Nabil.M nabil
coyote.F Togoniw
couteau.N wabaj
voir.V1 marufil
courir.V1 Nivumil
à ToT
lueur.M divuS
via

In [15]:
ding()